In [1]:
import pandas as pd
import numpy as np
import MySQLdb
from pandas.io import sql
from sqlalchemy import create_engine

In [2]:
with open("username.txt") as f:
    user = f.read()
with open("password.txt") as g:
    password = g.read()

In [3]:
engine = create_engine('mysql+mysqldb://' + user + ':' + password + '@127.0.0.1:3306/board_games?charset=utf8mb4')

In [4]:
user_ratings = pd.read_sql("SELECT * FROM user_ratings", engine)

In [5]:
user_ratings

,gameid,username,rating
0,30549,Torsten,10.0
1,30549,mitnachtKAUBO-I,10.0
2,30549,avlawn,10.0
3,30549,Mike Mayer,10.0
4,30549,Mease19,10.0
...,...,...,...
14064607,156089,himokl,4.0
14064608,156089,masterxavier,3.0
14064609,156089,tkzubaran,2.0
14064610,156089,Simplykoke,1.0


In [6]:
user_nrates = pd.DataFrame(user_ratings.username.value_counts())

In [7]:
user_nrates

,username
Doel,2474
leffe dubbel,2320
TomVasel,2106
JasonSaastad,2004
Hessu68,2000
...,...
Tankerone,1
LaMont7,1
LeapingDuck,1
Jazspinne,1


In [8]:
low_raters = user_nrates.loc[user_nrates['username'] < 10].index.tolist()

In [9]:
user_ratings_subset = user_ratings[~user_ratings.username.isin(low_raters)]

In [10]:
user_ratings_subset

,gameid,username,rating
0,30549,Torsten,10.0
1,30549,mitnachtKAUBO-I,10.0
2,30549,avlawn,10.0
3,30549,Mike Mayer,10.0
4,30549,Mease19,10.0
...,...,...,...
14064607,156089,himokl,4.0
14064608,156089,masterxavier,3.0
14064609,156089,tkzubaran,2.0
14064610,156089,Simplykoke,1.0


In [11]:
user_ratings_subset.drop_duplicates(inplace=True)

C:\Users\timco\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
multis = user_ratings_subset.loc[user_ratings_subset.duplicated(subset=["gameid", "username"], keep=False)]  # rows where (game, user) is duplicate
df_no_multis = user_ratings_subset.loc[~user_ratings_subset.duplicated(subset=["gameid", "username"], keep=False)] # rows where (game, user) is NOT duplicate

In [13]:
df_ratings = df_no_multis.copy()

In [15]:
means = multis.groupby(["gameid", "username"])["rating"].mean().reset_index()

In [16]:
means

,gameid,username,rating
0,1,BlueDane,7.500
1,1,Celtic,8.750
2,1,Hutchin,8.550
3,1,Iago71,8.250
4,1,Morganza,8.050
...,...,...,...
25157,286096,teddz,8.475
25158,287954,RudyJJ,8.500
25159,287954,makaveli172,7.450
25160,290448,jmarmyboy1,7.250


In [18]:
df_ratings = pd.concat([df_ratings, means])

In [19]:
df_ratings

,gameid,username,rating
0,30549,Torsten,10.000
1,30549,mitnachtKAUBO-I,10.000
2,30549,avlawn,10.000
3,30549,Mike Mayer,10.000
4,30549,Mease19,10.000
...,...,...,...
25157,286096,teddz,8.475
25158,287954,RudyJJ,8.500
25159,287954,makaveli172,7.450
25160,290448,jmarmyboy1,7.250


In [21]:
any(df_ratings[["username", "gameid"]].duplicated())

False

In [22]:
df_rating_pivot = df_ratings.groupby(['gameid', 'username'])['rating'].max().unstack()

In [23]:
df_rating_pivot

username,beastvol,mycroft,-=Yod@=-,-Johnny-,-Loren-,-LucaS-,-mIDE-,-mik-,-pj-,-snarf-,...,zzz2525,zzzabiss,zzzeagle,zzzk1,zzzkardel,zzzuzu,zzzvone,zzzxxxyyy,zzzzzane,Æleksandr Þræð
gameid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,4.0,NaN,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,8.0,NaN,7.0,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
type(df_rating_pivot)

pandas.core.frame.DataFrame

In [25]:
user_means = df_rating_pivot.mean(axis=1)

In [29]:
df_rating_pivot = df_rating_pivot.subtract(user_means, axis = 0)

MemoryError: 

In [28]:
df_rating_pivot

username,beastvol,mycroft,-=Yod@=-,-Johnny-,-Loren-,-LucaS-,-mIDE-,-mik-,-pj-,-snarf-,...,zzz2525,zzzabiss,zzzeagle,zzzk1,zzzkardel,zzzuzu,zzzvone,zzzxxxyyy,zzzzzane,Æleksandr Þræð
gameid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,4.0,NaN,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,8.0,NaN,7.0,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
